<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Using a SingleStoreDB Recursive CTE with London Underground data</h1>
    </div>
</div>

In [4]:
!pip cache purge --quiet

In [5]:
!pip install folium --quiet
!pip install geopandas --quiet

In [6]:
import folium
import geopandas as gpd

from folium import plugins
from singlestoredb import notebook as nb

In [7]:
connections_geojson_url = "https://raw.githubusercontent.com/VeryFatBoy/rcte-demo/refs/heads/main/datasets/northern_line_connections.geojson"
stations_geojson_url = "https://raw.githubusercontent.com/VeryFatBoy/rcte-demo/refs/heads/main/datasets/northern_line_stations.geojson"

connections_df = gpd.read_file(connections_geojson_url)
stations_df = gpd.read_file(stations_geojson_url)

In [8]:
London = [51.509865, -0.118092]

m = folium.Map(
    location = London,
    zoom_start = 12
)

folium.GeoJson(
    connections_df,
    name = "Northern Line",
    style_function = lambda feature: {
        "color" : "black",
        "weight" : 3
    }
).add_to(m)

folium.GeoJson(
    stations_df,
    name = "Northern Line Stations",
    popup = folium.GeoJsonPopup(fields = ["name"]),
    marker = folium.Marker(
        icon = folium.Icon(
            color = "black",
            icon = "train",
            icon_color = "white",
            prefix = "fa"
        )
    )
).add_to(m)

plugins.Fullscreen(
    position = "topright",
    title = "Fullscreen",
    title_cancel = "Exit",
    force_separate_button = True
).add_to(m)

html_content = m._repr_html_()

In [9]:
with nb.stage.open("map.html", "w") as st:
    st.write(html_content)

In [10]:
%%sql
CREATE DATABASE IF NOT EXISTS rcte_demo;

USE rcte_demo;

DROP TABLE IF EXISTS stations_route;
CREATE TABLE IF NOT EXISTS stations_route (
    station_from VARCHAR(22),
    station_to   VARCHAR(22),
    distance     NUMERIC(4,2)
);

INSERT INTO stations_route VALUES
('MORDEN','SOUTH WIMBLEDON',1.46),
('SOUTH WIMBLEDON','COLLIERS WOOD',1.14),
('COLLIERS WOOD','TOOTING BROADWAY',1.21),
('TOOTING BROADWAY','TOOTING BEC',1.11),
('TOOTING BEC','BALHAM',0.98),
('BALHAM','CLAPHAM SOUTH',1.16),
('CLAPHAM SOUTH','CLAPHAM COMMON',1.26),
('CLAPHAM COMMON','CLAPHAM NORTH',0.63),
('CLAPHAM NORTH','STOCKWELL',0.82),
('STOCKWELL','OVAL',1.37),
('OVAL','KENNINGTON',0.82),
('KENNINGTON','ELEPHANT & CASTLE',0.9),
('ELEPHANT & CASTLE','BOROUGH',0.85),
('BOROUGH','LONDON BRIDGE',0.68),
('LONDON BRIDGE','BANK',0.69),
('BANK','MOORGATE',0.82),
('MOORGATE','OLD STREET',0.69),
('OLD STREET','ANGEL',1.45),
('ANGEL','KINGS CROSS',1.38),
('KINGS CROSS','EUSTON',0.61),
('EUSTON','CAMDEN TOWN',1.72),
('KENNINGTON','WATERLOO',1.9),
('WATERLOO','EMBANKMENT',0.71),
('EMBANKMENT','CHARING CROSS',0.27),
('CHARING CROSS','LEICESTER SQUARE',0.47),
('LEICESTER SQUARE','TOTTENHAM COURT ROAD',0.4),
('TOTTENHAM COURT ROAD','GOODGE STREET',0.63),
('GOODGE STREET','WARREN STREET',0.47),
('WARREN STREET','EUSTON',0.58),
('EUSTON','MORNINGTON CRESCENT',0.82),
('MORNINGTON CRESCENT','CAMDEN TOWN',0.64),
('CAMDEN TOWN','KENTISH TOWN',1.15),
('KENTISH TOWN','TUFNELL PARK',0.8),
('TUFNELL PARK','ARCHWAY',0.89),
('ARCHWAY','HIGHGATE',1.71),
('HIGHGATE','EAST FINCHLEY',1.64),
('EAST FINCHLEY','FINCHLEY CENTRAL',2.43),
('FINCHLEY CENTRAL','MILL HILL EAST',1.52),
('FINCHLEY CENTRAL','WEST FINCHLEY',1.14),
('WEST FINCHLEY','WOODSIDE PARK',0.98),
('WOODSIDE PARK','TOTTERIDGE & WHETSTONE',1.51),
('TOTTERIDGE & WHETSTONE','HIGH BARNET',2.48),
('CAMDEN TOWN','CHALK FARM',0.85),
('CHALK FARM','BELSIZE PARK',1.11),
('BELSIZE PARK','HAMPSTEAD',1.21),
('HAMPSTEAD','GOLDERS GREEN',2.35),
('GOLDERS GREEN','BRENT CROSS',1.56),
('BRENT CROSS','HENDON CENTRAL',1.11),
('HENDON CENTRAL','COLINDALE',2.11),
('COLINDALE','BURNT OAK',1.32),
('BURNT OAK','EDGWARE',1.47);


1 rows affected.

51 rows affected.

++
||
++
++

In [11]:
%%sql
USE rcte_demo;

WITH RECURSIVE possible_route AS (
    SELECT sr.station_to,
           CONCAT (sr.station_from, '->', sr.station_to) AS route,
           sr.distance
    FROM stations_route sr
    WHERE sr.station_from = 'MORDEN'
UNION ALL
    SELECT sr.station_to,
           CONCAT (pr.route, '->', sr.station_to) AS route,
           pr.distance + sr.distance
    FROM possible_route pr
INNER JOIN stations_route sr
    ON sr.station_from = pr.station_to
)
SELECT pr.route,
       pr.distance
FROM possible_route pr
WHERE pr.station_to = 'EUSTON'
ORDER BY pr.distance;

2 rows affected.

route,distance
MORDEN->SOUTH WIMBLEDON->COLLIERS WOOD->TOOTING BROADWAY->TOOTING BEC->BALHAM->CLAPHAM SOUTH->CLAPHAM COMMON->CLAPHAM NORTH->STOCKWELL->OVAL->KENNINGTON->WATERLOO->EMBANKMENT->CHARING CROSS->LEICESTER SQUARE->TOTTENHAM COURT ROAD->GOODGE STREET->WARREN STREET->EUSTON,17.39
MORDEN->SOUTH WIMBLEDON->COLLIERS WOOD->TOOTING BROADWAY->TOOTING BEC->BALHAM->CLAPHAM SOUTH->CLAPHAM COMMON->CLAPHAM NORTH->STOCKWELL->OVAL->KENNINGTON->ELEPHANT & CASTLE->BOROUGH->LONDON BRIDGE->BANK->MOORGATE->OLD STREET->ANGEL->KINGS CROSS->EUSTON,20.03
